In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import sys

In [4]:
path = "../../data/reg_dataset.csv"
df = pd.read_csv(path)
data = df
data.head

<bound method NDFrame.head of       Normalized_BB    Target
0        -20.551918 -0.106135
1        -19.632087  0.135392
2        -22.900646  0.061923
3          5.600050  0.137273
4          4.829646  0.220390
...             ...       ...
4791     137.018761  0.085582
4792     123.893304  0.032600
4793     127.591140  0.038679
4794      98.292272 -0.020345
4795      98.291929 -0.003049

[4796 rows x 2 columns]>